In [9]:
import pickle
from glob import glob
import pandas as pd
from bert_serving.client import BertClient

In [10]:
bc = BertClient()

In [11]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


In [19]:
books = glob('books/*.txt')

for book in books:
    txt = open(book, 'r').read()
    
    sentences = split_into_sentences(txt)
    pickle.dump(sentences, open(book.rstrip('.txt') + '_sentences.pkl', 'wb'))
    
    embeddings = bc.encode(sentences)
    pickle.dump(embeddings, open(book.rstrip('.txt') + '_embeddings.pkl', 'wb'))
    
    print(book)
    print(f'\tNumber of sentences: {len(sentences)}')
    print(f'\tLonguest sentence: {max([len(s.split(" ")) for s in sentences])}')
    print(f'\tEmbeddings: {embeddings.shape}')


books/the_great_gatsby.txt
	Number of sentences: 5
	Longuest sentence: 48
	Embeddings: (5, 1024)
books/hobbit.txt
	Number of sentences: 5
	Longuest sentence: 21
	Embeddings: (5, 1024)
books/mobydick.txt
	Number of sentences: 5
	Longuest sentence: 76
	Embeddings: (5, 1024)
books/dune.txt
	Number of sentences: 5
	Longuest sentence: 33
	Embeddings: (5, 1024)
books/1984.txt
	Number of sentences: 5
	Longuest sentence: 41
	Embeddings: (5, 1024)
